In [89]:
import numpy as np

In [90]:
#load_data
movie=[]
std=[]
with open ("/Users/ivanlin328/Desktop/CSE 250A/HW7/hw7_movies.txt",'r') as movies:
    for line in movies.readlines():
        movie.append(line.strip())
with open ("/Users/ivanlin328/Desktop/CSE 250A/HW7/hw7_ids.txt",'r') as students:
    for line in students.readlines():
        std.append(line.strip())

ratings=np.loadtxt("/Users/ivanlin328/Desktop/CSE 250A/HW7/hw7_ratings.txt",dtype='str')
print(ratings.shape)

(525, 60)


In [91]:
import pandas as pd
rating=np.zeros(ratings.shape[1])
for i in range(ratings.shape[1]):
    count1=0
    count2=0
    for j in range(ratings.shape[0]):
        if ratings[j,i]!='?':
            count2+=1
            if ratings[j,i]=="1":
                count1+=1     
    if count2>0:
        ratio=count1/count2
        rating[i]=ratio
data=pd.DataFrame({"Movies": movie,"Popularity ratings":rating})
data=data.sort_values(by="Popularity ratings",ascending=False)
data=data.reset_index(drop=True)
print(data)

                                               Movies  Popularity ratings
0                                        Interstellar            0.948187
1                                     The Dark Knight            0.941358
2                                          The Matrix            0.940120
3                                           Inception            0.935976
4                                              WALL-E            0.923483
5                                Jurassic Park (1993)            0.916084
6                                  The Imitation Game            0.910569
7                             Guardians of the Galaxy            0.907609
8                                      Doctor Strange            0.900000
9                             The Wolf of Wall Street            0.898990
10                             Avengers: Infinity War            0.891304
11                                        The Martian            0.888489
12  The Lord of the Rings: The Fellows

In [92]:
prob_z= np.loadtxt("/Users/ivanlin328/Desktop/CSE 250A/HW7/hw7_probZ_init.txt",dtype=float)
prob_r=np.loadtxt("/Users/ivanlin328/Desktop/CSE 250A/HW7/hw7_probR_init.txt")
print(prob_z.shape[0])
print(prob_r.shape)

4
(60, 4)


In [93]:
def likelihood(t, prob_z, prob_r,k):
    likeli = 0
    for i in range(k):
        j_rec = np.asarray(ratings[t, :] == "1").nonzero()  
        j_notrec = np.asarray(ratings[t, :] == "0").nonzero()
        likeli += prob_z[i] * np.prod(prob_r[j_rec, i]) * np.prod(1 - prob_r[j_notrec, i])
    return likeli

def log_likelihood(prob_z, prob_r,k):
    log_likeli = 0
    T = ratings.shape[0]
    for t in range(T):
        user_likelihood = likelihood(t, prob_z, prob_r, k)
        log_likeli += np.log(user_likelihood)
    return log_likeli / T
 

In [94]:
def estep_numerator(i, t, prob_z, prob_r, ratings):
    j_rec = np.asarray(ratings[t, :] == "1").nonzero()
    j_notrec = np.asarray(ratings[t, :] == "0").nonzero()
    return prob_z[i] * np.prod(prob_r[j_rec, i]) * np.prod(1 - prob_r[j_notrec, i])
  
def estep_denominator(t, prob_z, prob_r, ratings, k):
    denom = 0
    j_rec = np.asarray(ratings[t, :] == "1").nonzero()
    j_notrec = np.asarray(ratings[t, :] == "0").nonzero()   
    for i in range(k):
        denom += prob_z[i] * np.prod(prob_r[j_rec, i]) * np.prod(1 - prob_r[j_notrec, i])
    return denom


In [95]:
def m_step_prob_r(i,j,posteriors,prob_r):
    #seen part
    t_seen=np.asarray(ratings[:,j] == "1").nonzero()
    sum_seen= np.sum(posteriors[i,t_seen])
    #unseen part
    t_unseen=np.asarray(ratings[:,j] == "?").nonzero()
    sum_unseen=np.sum(posteriors[i,t_unseen])*prob_r[j,i]
    return sum_seen+sum_unseen
    

In [96]:
# EM Algorithm
N = 256
target = [0, 1, 2, 4, 8, 16, 32, 64, 128, 256]
toprint = []
log_likelihoods = []
T = ratings.shape[0]
k = prob_z.shape[0]
M = ratings.shape[1]
posteriors = np.zeros([k,T], dtype=float)
new_prob_z = np.zeros(k)
new_prob_r = np.zeros([M,k])

for i in range(N + 1):
    if i in target:
        toprint.append(i)
        log_likelihoods.append(log_likelihood(prob_z, prob_r, k))
    
    for t in range(T):
        e_denom = estep_denominator(t, prob_z, prob_r, ratings, k)
        for i in range(k):
            posteriors[i,t] = estep_numerator(i, t, prob_z, prob_r, ratings) / e_denom
    
    for i in range(k):
        new_prob_z[i] = np.sum(posteriors[i,:]) / T
        for j in range(M):
            new_prob_r[j,i] = m_step_prob_r(i, j, posteriors, prob_r) / np.sum(posteriors[i,:])
    
    prob_z = new_prob_z.copy()  
    prob_r = new_prob_r.copy()

print(toprint)
print(log_likelihoods)

[0, 1, 2, 4, 8, 16, 32, 64, 128, 256]
[-28.092370113661485, -16.493378113831568, -14.561167073337495, -13.625096360195535, -13.23329988412404, -13.06644206787089, -12.99603729576008, -12.992546414601215, -12.992246667930614, -12.992232245896068]


In [97]:
from prettytable import PrettyTable
print("Table:")
x=PrettyTable()
x.add_column("Iteration:",toprint)
x.add_column("Log-logliklihood",log_likelihoods)
print(x)

Table:
+------------+---------------------+
| Iteration: |   Log-logliklihood  |
+------------+---------------------+
|     0      | -28.092370113661485 |
|     1      | -16.493378113831568 |
|     2      | -14.561167073337495 |
|     4      | -13.625096360195535 |
|     8      |  -13.23329988412404 |
|     16     |  -13.06644206787089 |
|     32     |  -12.99603729576008 |
|     64     | -12.992546414601215 |
|    128     | -12.992246667930614 |
|    256     | -12.992232245896068 |
+------------+---------------------+


In [98]:
studId=[]
with open ("/Users/ivanlin328/Desktop/CSE 250A/HW7/hw7_ids.txt",'r') as id:
    for line in id.readlines():
        studId.append(line.strip())
PID="A69036794"
for i in range(len(studId)):
    if studId[i]==PID:
        index=i
print(index)

72


In [100]:
my_data=ratings[index,:]
my_unseen_data=np.asarray(my_data == "?").nonzero()[0]

expected_ratings=[]
unseen_movies=[]
for l in my_unseen_data:
    unseen_movies.append(movie[l])
    expecting_rating=0
    for i in range(k):
        e_step= estep_numerator(i,index, prob_z, prob_r, ratings)/estep_denominator(index, prob_z, prob_r, ratings,k)
        m_step= m_step_prob_r(i,l,posteriors,prob_r)/np.sum(posteriors[i,:])
        expecting_rating+=e_step*m_step
    expected_ratings.append(expecting_rating) 
      
datas=pd.DataFrame({"Movies":unseen_movies ,"Popularity ratings":expected_ratings})
datas=datas.sort_values(by="Popularity ratings",ascending=False)
datas=datas.reset_index(drop=True) 
print(datas)
         
        
        
    

                   Movies  Popularity ratings
0              The Matrix            0.916831
1               Inception            0.895516
2      The Imitation Game            0.874394
3              La La Land            0.820911
4                    Coco            0.794524
5              Ex Machina            0.781116
6   2001: A Space Odyssey            0.768758
7       Blade Runner 2049            0.762830
8                 Get Out            0.717651
9                   Moana            0.679009
10     Terminator Genisys            0.672817
11           Man of Steel            0.664476
12                   Tron            0.654687
13     The Shape of Water            0.602985
14              Rogue One            0.556266
15            World War Z            0.545665
16     The Last Airbender            0.531576
17       Star Trek Beyond            0.528702
18           Wonder Woman            0.520161
19                   Solo            0.517823
